<a href="https://colab.research.google.com/github/nakkata/dev_tool/blob/main/chat_gpt_database_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# チャットGPTで社内ドキュメントの解析エンジンを作る

In [1]:
# ChatGPTで解析
!pip install openai==0.28.0
!pip install pandas
!pip install transformers==4.32.1
!pip install langchain==0.0.279
#!pip install beautifulsoup4==4.11.2
!pip install textract
#!pip install codecs
#!pip install six==1.13.0
!pip install requests
!pip install pypdf
!pip install tiktoken
!pip install faiss-gpu

# Webアプリ化
!pip install streamlit==1.20.0 --quiet
!pip install pyngrok==4.1.1 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 9.5 MB/s eta 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# ChatGPTで解析
import os
import pandas as pd
import requests
import textract
import codecs
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

# Webアプリ化
import streamlit as st
from pyngrok import ngrok
import openai

In [ ]:
#ドライブをマウント
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
%%writefile app.py
# 以下を「app.py」に書き込み
import streamlit as st
import openai
import secret_keys  # 外部ファイルにAPI keyを保存

openai.api_key = secret_keys.openai_api_key

system_prompt = """
あなたは優秀なアシスタントです。
質問に対して適切な対処法のアドバイスを行ってください。
以下のようなことを聞かれても、絶対に答えないでください。

* 旅行
* 料理
* 芸能人
* 映画
* 科学
* 歴史
"""

# st.session_stateを使いメッセージのやりとりを保存
if "messages" not in st.session_state:
    st.session_state["messages"] = [
        {"role": "system", "content": system_prompt}
        ]

# チャットボットとやりとりする関数
def communicate():
    messages = st.session_state["messages"]

    user_message = {"role": "user", "content": st.session_state["user_input"]}
    messages.append(user_message)

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    bot_message = response["choices"][0]["message"]
    messages.append(bot_message)

    st.session_state["user_input"] = ""  # 入力欄を消去


# インターネット上からマニュアルファイルをダウンロード（BuffaloTerastation)
url = 'https://support2.mexcbt.mext.go.jp/files/MEXCBT簡易マニュアル_児童編（OPE）_第4版.pdf'
response = requests.get(url)

loader = PyPDFLoader("/content/sample_document1.pdf")
pages = loader.load_and_split()

chunks = pages
print("step2")


# Get embedding model
embeddings = OpenAIEmbeddings()


#  vector databaseの作成
db = FAISS.from_documents(chunks, embeddings)

query = "自治体コード入力画面とは？"
# FAISSに対して検索。検索は文字一致ではなく意味一致で検索する(Vector, Embbeding)
docs = db.similarity_search(query)
docs # ここで関係のありそうなデータが返ってきていることを確認できる

print("step7")
# 得られた情報から回答を導き出すためのプロセスを以下の4つから選択する。いずれもProsとConsがあるため、適切なものを選択する必要がある。
# staffing ... 得られた候補をそのままインプットとする
# map_reduce ... 得られた候補のサマリをそれぞれ生成し、そのサマリのサマリを作ってインプットとする
# map_rerank ... 得られた候補にそれぞれスコアを振って、いちばん高いものをインプットとして回答を得る
# refine  ... 得られた候補のサマリを生成し、次にそのサマリと次の候補の様裏を作ることを繰り返す
chain = load_qa_chain(OpenAI(temperature=0.1,max_tokens=1000), chain_type="stuff")
# p305に記載
#query = "ドライブのランプが赤色に点滅しているが、これは何が原因か？"
# p134に記載
#query = "どの様な時にメイン機が異常だと判断をしますか？"
query = "ログイン方法を教えてください。"
docs = db.similarity_search(query)
print("step8")
# langchainを使って検索
chain.run(input_documents=docs, question=query)

from IPython.display import display
import ipywidgets as widgets

print("step9")
# vectordbをretrieverとして使うconversation chainを作成します。これはチャット履歴の管理も可能にします。
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.1), db.as_retriever())

chat_history = []

# print("step10")
# def on_submit(_):
#     query = input_box.value
#     input_box.value = ""
#
#     if query.lower() == 'exit':
#         print("Thank you for using the State of the Union chatbot!")
#         return
#
#     result = qa({"question": query, "chat_history": chat_history})
#     chat_history.append((query, result['answer']))

# ユーザーインターフェイスの構築
st.title(" 「英語講師」ボット")
st.image("03_english.png")
st.write("Let's enjoy English!")

user_input = st.text_input("メッセージを入力してください。", key="user_input", on_change=communicate)

if st.session_state["messages"]:
    messages = st.session_state["messages"]

    for message in reversed(messages[1:]):  # 直近のメッセージを上に
        speaker = "🙂"
        if message["role"]=="assistant":
            speaker="🤖"

        st.write(speaker + ": " + message["content"])

Overwriting app.py


In [3]:

# ここにOpenAIから取得したキーを設定します。
os.environ["OPENAI_API_KEY"] = ''

In [ ]:
# 社内用で使う時はここをコメントアウト
## proxyの設定
#os.environ["HTTP_PROXY"] = "http://10.40.210.120:8080"
#os.environ["HTTPS_PROXY"] = "http://10.40.210.120:8080"

In [4]:

# LLMの設定
llm = OpenAI(model_name="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
### オンラインドキュメントから情報を取得する

# インターネット上からマニュアルファイルをダウンロード（BuffaloTerastation)
url = 'https://support2.mexcbt.mext.go.jp/files/MEXCBT簡易マニュアル_児童編（OPE）_第4版.pdf'
response = requests.get(url)

# Ensure that the request was successful
if response.status_code == 200:
    # Save the content of the response as a PDF file
    with open('sample_document1.pdf', 'wb') as file:
        file.write(response.content)
else:
    print("Error: Unable to download the PDF file. Status code:", response.status_code)

In [5]:
### ローカルドキュメントから情報を取得する

loader = PyPDFLoader("/content/test2.pdf")
pages = loader.load_and_split()

In [6]:
chunks = pages
print("step2")

step2


In [7]:

# Get embedding model
embeddings = OpenAIEmbeddings()


In [8]:

#  vector databaseの作成
db = FAISS.from_documents(chunks, embeddings)

INFO:faiss.loader:Loading faiss with AVX2 support.
2023-10-19 23:09:14.115 INFO    faiss.loader: Loading faiss with AVX2 support.
INFO:faiss.loader:Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2023-10-19 23:09:14.121 INFO    faiss.loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
INFO:faiss.loader:Loading faiss.
2023-10-19 23:09:14.127 INFO    faiss.loader: Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
2023-10-19 23:09:14.153 INFO    faiss.loader: Successfully loaded faiss.


In [9]:
query = "遊技球について"
# FAISSに対して検索。検索は文字一致ではなく意味一致で検索する(Vector, Embbeding)
docs = db.similarity_search(query)
docs # ここで関係のありそうなデータが返ってきていることを確認できる


[Document(page_content='別表第六 アレンジボール遊技機に係る技術上の規格（第六条関係）\n- 1 -規 則  解釈基準  \n(1) \nイ \nロ (ｲ)\n(ﾛ)\n(ﾊ)\n(ｲ)\n(ﾛ)\n(ﾊ)\n(ﾆ)\n(ﾎ)性能に関する規格 \n 遊技機を作動させるための遊技メダル等の投入に係る遊技機の性能に関する規格は、次\nのとおりとする。 \n 規定数は、一回の遊技（規定数の遊技メダル等の投入をした時から当該遊技メダル等に\n係る遊技の結果が得られる時までの間における遊技球を用いて行う遊技をいう。以下この\n表及び次表において同じ。 ）につき、遊技メダルにあつては三枚であり、遊技球にあつては\n十五個であること。 \n 規定数の遊技メダル等の投入ごとに、十五個又は十六個の遊技球を使用して遊技をさせ\nることができるものであること。 \n 規定数の遊技メダル等の投入をした場合においては、その投入をした時から当該遊技メ\nダル等に係る遊技の結果が得られる時までの間は、新たに遊技メダル等の投入をすることができるものでないこと。 \n発射装置の性能に関する規格は、次のとおりとする。 \n遊技球を一個ずつ発射することができるものであること。 \n 電気的動力により作動する発射装置を有する遊技機にあつては、遊技球を連続して発射\nすることができるものでないこと。 \n一分間に百個を超える遊技球を発射することができるものでないこと。 \n 遊技球の試射試験を十時間行つた場合において、 （イ） 、（ロ）及び（ハ）に掲げる性能が\n不変であるものであること。 \n 遊技球の試射試験を十時間行つた場合において、その間、遊技盤上の遊技球の位置を確\n認し、かつ、調整することができるものであること。', metadata={'source': '/content/test2.pdf', 'page': 0}),
 Document(page_content='別表第六 アレンジボール遊技機に係る技術上の規格（第六条関係）\n- 2 -規 則  解釈基準  \n ハ  \n(ｲ)\n(ﾛ)\n(ﾊ)\n(ﾆ)\n(ﾎ)(ﾍ)(ﾄ)\n(ﾁ)遊技メダル等の獲得に係る遊技機の性能に関する規格は、次のとおりとする。 \n 入賞があつた場合に獲得することができる遊技メ

In [10]:
print("step7")
# 得られた情報から回答を導き出すためのプロセスを以下の4つから選択する。いずれもProsとConsがあるため、適切なものを選択する必要がある。
# staffing ... 得られた候補をそのままインプットとする
# map_reduce ... 得られた候補のサマリをそれぞれ生成し、そのサマリのサマリを作ってインプットとする
# map_rerank ... 得られた候補にそれぞれスコアを振って、いちばん高いものをインプットとして回答を得る
# refine  ... 得られた候補のサマリを生成し、次にそのサマリと次の候補の様裏を作ることを繰り返す
chain = load_qa_chain(OpenAI(temperature=0.1,max_tokens=1000), chain_type="stuff")
# p305に記載
#query = "ドライブのランプが赤色に点滅しているが、これは何が原因か？"
# p134に記載
#query = "どの様な時にメイン機が異常だと判断をしますか？"
query = "図柄の組み合わせ"
docs = db.similarity_search(query)
print("step8")


step7
step8


In [11]:
docs

[Document(page_content='別表第六 アレンジボール遊技機に係る技術上の規格（第六条関係）\n- 1 -規 則  解釈基準  \n(1) \nイ \nロ (ｲ)\n(ﾛ)\n(ﾊ)\n(ｲ)\n(ﾛ)\n(ﾊ)\n(ﾆ)\n(ﾎ)性能に関する規格 \n 遊技機を作動させるための遊技メダル等の投入に係る遊技機の性能に関する規格は、次\nのとおりとする。 \n 規定数は、一回の遊技（規定数の遊技メダル等の投入をした時から当該遊技メダル等に\n係る遊技の結果が得られる時までの間における遊技球を用いて行う遊技をいう。以下この\n表及び次表において同じ。 ）につき、遊技メダルにあつては三枚であり、遊技球にあつては\n十五個であること。 \n 規定数の遊技メダル等の投入ごとに、十五個又は十六個の遊技球を使用して遊技をさせ\nることができるものであること。 \n 規定数の遊技メダル等の投入をした場合においては、その投入をした時から当該遊技メ\nダル等に係る遊技の結果が得られる時までの間は、新たに遊技メダル等の投入をすることができるものでないこと。 \n発射装置の性能に関する規格は、次のとおりとする。 \n遊技球を一個ずつ発射することができるものであること。 \n 電気的動力により作動する発射装置を有する遊技機にあつては、遊技球を連続して発射\nすることができるものでないこと。 \n一分間に百個を超える遊技球を発射することができるものでないこと。 \n 遊技球の試射試験を十時間行つた場合において、 （イ） 、（ロ）及び（ハ）に掲げる性能が\n不変であるものであること。 \n 遊技球の試射試験を十時間行つた場合において、その間、遊技盤上の遊技球の位置を確\n認し、かつ、調整することができるものであること。', metadata={'source': '/content/test2.pdf', 'page': 0}),
 Document(page_content='別表第六 アレンジボール遊技機に係る技術上の規格（第六条関係）\n- 2 -規 則  解釈基準  \n ハ  \n(ｲ)\n(ﾛ)\n(ﾊ)\n(ﾆ)\n(ﾎ)(ﾍ)(ﾄ)\n(ﾁ)遊技メダル等の獲得に係る遊技機の性能に関する規格は、次のとおりとする。 \n 入賞があつた場合に獲得することができる遊技メ

In [12]:

# langchainを使って検索
chain.run(input_documents=docs, question=query)

' 四以下の数の図柄の組合せで入賞に係るものの数は、六以上であること。'

In [13]:
from IPython.display import display
import ipywidgets as widgets

In [14]:

print("step9")
# vectordbをretrieverとして使うconversation chainを作成します。これはチャット履歴の管理も可能にします。
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.1), db.as_retriever())

step9


In [15]:
chat_history = []

In [16]:
print("step10")
def on_submit(_):
    query = input_box.value
    input_box.value = ""

    if query.lower() == 'exit':
        print("Thank you for using the State of the Union chatbot!")
        return

    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))

    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))

print("Welcome to the Transformers chatbot! Type 'exit' to stop.")


step10
Welcome to the Transformers chatbot! Type 'exit' to stop.


In [20]:
input_box = widgets.Text(placeholder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)

Text(value='', placeholder='Please enter your question:')

HTML(value='<b>User:</b> あ')

HTML(value='<b><font color="blue">Chatbot:</font></b>  I\'m sorry, I don\'t know the answer to that question.'…

HTML(value='<b>User:</b> アレンジぼ')

HTML(value='<b><font color="blue">Chatbot:</font></b>  I don\'t know.')

HTML(value='<b>User:</b> アレンジボール')

HTML(value='<b><font color="blue">Chatbot:</font></b>  アレンジボールは、電気的動力により作動する発射装置を有する遊技機で、一回の遊技では3枚の遊技メダルを投入し、1…

HTML(value='<b>User:</b> 遊技球')

HTML(value='<b><font color="blue">Chatbot:</font></b>  一回の遊技では15個の遊技球を使用します。')